In [ ]:
import boto3
import numpy as np
import os
import pandas as pd
import scanpy as sc
import tarfile
from cellxgene_schema.write_labels import AnnDataLabelAppender
from urllib.parse import quote
from cellxgene_ontology_guide.ontology_parser import OntologyParser
ontology_parser = OntologyParser(schema_version="v6.0.0")
from cellxgene_ontology_guide.supported_versions import CXGSchema, load_supported_versions
import requests
from io import BytesIO

s3client = boto3.client('s3')

In [ ]:
def download_files(s_dir):
    mx_h5 = f'{s_dir}/count/sample_filtered_feature_bc_matrix.h5'
    metrics_csv = f'{s_dir}/metrics_summary.csv'
    cri_tar = f'{s_dir}/count/crispr_analysis.tar.gz'
    
    for file_path in [mx_h5, metrics_csv, cri_tar]:
        f = file_path.split('/')[-1]
        s3client.download_file(bucket_name, file_path, f)


def custom_var_to_obs(adata):
    moved = []
    for gene_index in adata.var[
        (adata.var['feature_types'] == 'Gene Expression') &
        (adata.var['gene_ids'].str.startswith('ENS') == False)
    ].index:
        gene_id = adata.var.loc[gene_index]['gene_ids']
        gene_values = adata.X[:, adata.var.index.get_loc(gene_index)]
        adata.obs[gene_id] = gene_values.A.flatten() if hasattr(gene_values, 'A') else gene_values.toarray().flatten().tolist()
        moved.append(gene_id)

    adata.var.set_index('gene_ids', inplace=True)
    var_to_keep = [i for i in adata.var.index if i not in moved]
    adata = adata[:, var_to_keep]
    adata.var = adata.var.replace('', np.nan).dropna(axis=1, how='all')

    return adata


def gather_crispr():
    '''
    obtain guide assignement from 10x cellranger output files
    currently will update 1-Jun and 2-Jun to JUN-1 and JUN-2, respectively

    return df: dataframe containing crispr guide calling
    '''
    tar = tarfile.open('crispr_analysis.tar.gz', 'r:gz')
    f = tar.extractfile('protospacer_calls_per_cell.csv')
    df = pd.read_csv(f).rename(columns={'num_umis':'num_umis_guide_id'})
    tar.close()
    df['genetic_perturbation_guide_id'] = df['feature_call'].apply(lambda x: x.replace('|',' || '))
    df['genetic_perturbation_guide_id'] = df['genetic_perturbation_guide_id'].str.replace('1-Jun', 'JUN-1')
    df['genetic_perturbation_guide_id'] = df['genetic_perturbation_guide_id'].str.replace('2-Jun', 'JUN-2')
    df = df[['cell_barcode','genetic_perturbation_guide_id','num_umis_guide_id']].set_index('cell_barcode')

    return df


def gather_metrics(samp):
    df = pd.read_csv('metrics_summary.csv')
    df['Metric'] = df.apply(lambda x: f"{x['Metric Name']}, {x['Library Type']}, {x['Category'].replace('Cells','Sample')}", axis=1)

    probe_barcodes = ' || '.join(df[
        (df['Metric Name'] == 'Sample ID') &
        (df['Metric Value'] == samp)
    ]['Group Name'].unique())

    df = df[
        (df['Grouped By'].isin(['Fastq ID','Probe barcode ID']) == False) &
        (df['Category'].isin(['Library','Cells']))
    ]
    df = df[['Metric','Metric Value']].set_index('Metric').transpose()

    for c in df.columns:
        v = df[c].iloc[0]
        if v.endswith('%'):
            df[c] = df[c].apply(lambda x: float(x.rstrip('%')) / 100)
        elif '%' in v:
            df.drop(columns=c, inplace=True)
        else:
            df[c] = df[c].apply(lambda x: int(x.replace(',','')))

    df['Probe barcode IDs'] = probe_barcodes

    keep = {
        'Confidently mapped reads in cells, Gene Expression, Sample': 'fraction_mapped_reads_in_cells_gex',
        'Median genes per cell, Gene Expression, Sample': 'median_genes_per_cell',
        'Cells, CRISPR Guide Capture, Sample': 'cell_count_cri',
        'Cells, Gene Expression, Sample': 'cell_count_gex',
        'Valid GEM barcodes, CRISPR Guide Capture, Library': 'fraction_valid_barcodes_cri',
        'Valid GEM barcodes, Gene Expression, Library': 'fraction_valid_barcodes_gex',
        'Reads confidently mapped to probe set, Gene Expression, Sample': 'frac_reads_mapped_gex',
        'Confidently mapped to genome, Gene Expression, Library': 'frac_reads_mapped_gex',
        'Mean reads per cell, Gene Expression, Sample': 'mean_reads_per_cell_gex'
    }

    df = df[[f for f in keep.keys() if f in df.columns]].rename(columns=keep)

    return df


def cxg_add_labels(adata):
    adata.obs['cell_type_ontology_term_id'] = 'unknown'
    labeler = AnnDataLabelAppender(adata)
    labeler._add_labels()
    adata.obs.drop(columns=['cell_type_ontology_term_id','cell_type'],inplace=True)

    schema_v = labeler.schema_version
    adata.uns['schema_version'] = schema_v
    adata.uns['schema_reference'] = labeler._build_schema_reference_url(schema_v)


def add_guide_metadata(adata, sheet, guide_gid):
    '''
    Add guide metadata into adata.uns from Lattice wrangling sheet
    
    :param obj adata: the anndata object that is being transformed into the curated matrix
    :param obj guide_df: the dataframe containing guide metadata from wrangling sheet
    
    :returns obj adata: modified adata to contain guide metadata
    '''
    url = f'https://docs.google.com/spreadsheets/d/{sheet}/export?format=csv&gid={guide_gid}'
    response = requests.get(url)
    guide_df = pd.read_csv(BytesIO(response.content), comment="#", dtype=str)
    genetic_perturbations = {}
    
    for row in guide_df.itertuples():
        genetic_perturbations[row.guide_id] = {}
        genetic_perturbations[row.guide_id]['role'] = 'targeting' if row.guide_role == 'Targeting a Gene' else 'control'
        genetic_perturbations[row.guide_id]['protospacer_sequence'] = row.guide_protospacer
        genetic_perturbations[row.guide_id]['protospacer_adjacent_motif'] = row.guide_PAM
        genetic_perturbations[row.guide_id]['target_genomic_regions'] = [str(row.chromosome).replace("chr","") +
                                                                         ":" + str(row.start) +
                                                                         ":" + str(row.end) + 
                                                                         ":" + str(row.strand)]
        if not pd.isna(row.overlapping_gene_ids):
            genetic_perturbations[row.guide_id]['target_features'] = {}
            for i in range(len(row.overlapping_gene_ids.split(";"))):
                genetic_perturbations[row.guide_id]['target_features'][row.overlapping_gene_ids.split(";")[i]] = row.overlapping_gene_names.split(";")[i]
                                                                             
            
    adata.uns['genetic_perturbations'] = genetic_perturbations
    
    return adata


def determine_perturbation_strategy(adata):
    '''
    Assess feature_call from protospacer_calls_per_cell.csv, where if all guides
    assigned to a single cell are all control, then 'control'. Otherwise, it is "no perturbations"
    if no guids or one of the following if targeting:
        - "CRISPR activation screen"
        - "CRISPR interference screen"
        - "CRISPR knockout mutant"
        - "CRISPR knockout screen"
    
    :param obj adata: the anndata object that is being transformed into the curated matrix

    :returns obj adata: modified adata to contain perturbation_strategy as cell metadata
    '''
    adata.obs['genetic_perturbation_strategy_calculated'] = adata.obs['genetic_perturbation_guide_id']
    adata.obs['genetic_perturbation_strategy_calculated'] = adata.obs['genetic_perturbation_strategy_calculated'].apply(
        lambda x: x.split(' || ') if pd.notna(x) else 'no perturbations'
    )
    adata.obs['genetic_perturbation_strategy_calculated'] = adata.obs['genetic_perturbation_strategy_calculated'].apply(
        lambda x: [adata.uns['genetic_perturbations'][i]['role'] for i in x] if isinstance(x, list)
            else x        
    )
    adata.obs['genetic_perturbation_strategy_calculated'] = adata.obs['genetic_perturbation_strategy_calculated'].apply(
         lambda x: 'control' if isinstance(x, list) and 'targeting' not in set(x)
            else x
    )
    adata.obs.loc[adata.obs['genetic_perturbation_strategy_calculated']=='control', 'genetic_perturbation_strategy'] = 'control'
    adata.obs.loc[adata.obs['genetic_perturbation_strategy_calculated']=='no perturbations', 'genetic_perturbation_strategy'] = 'no perturbations'
    adata.obs.drop(columns=['genetic_perturbation_strategy_calculated'], inplace=True)
    
    return adata


def add_library_id(adata, sheet_id, pooled_gid, sample):
    '''
    load in pooled library template and determine which library "pool_name" that this subsample was derived from.
    This is specific to 10x assays, and will have to be re-evaluated for plate assays

    :param obj adata: the anndata object that is being transformed into the curated matrix
    :param str sheet_id: the gid of the google sheet for the wrangling template
    :param str pooled_gid: the gid of the google sheet template tab for "pooled library template"
    :param str sample: the sampleID of this subsample

    :returns obj adata: modified adata that contains library idenfier
    '''
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={pooled_gid}'
    pooled_df = pd.read_csv(url, comment='#', index_col=False).dropna(axis=1, how='all')
    library_id = pooled_df.loc[(pooled_df['sample_name'] == sample) & (pooled_df['library type']=='GEX'), 'pool_name'][0]
    adata.obs['library_name'] = library_id

    return adata

In [ ]:
#read in the sample metadata
sheet = '1YVhCswWQD6YsC_oqLCejzseraW_Xxi6BUp1_MXOsUII'
gid = '1236594312'
url = f'https://docs.google.com/spreadsheets/d/{sheet}/export?format=csv&gid={gid}'
response = requests.get(url)
sample_df = pd.read_csv(BytesIO(response.content), comment="#", dtype=str).dropna(axis=1,how='all')

In [ ]:
# Mapping ontologies

col_ont_map = {
    'organism':'NCBITaxon',
    'sex':'PATO',
    'self_reported_ethnicity':'HANCESTRO',
    'disease':'MONDO',
    'assay':'EFO',
    'development_stage':{'NCBITaxon:6239':'WBls', # C. Elegans
                         'NCBITaxon:7227':'FBdv', # Drosophila
                         'NCBITaxon:10090':'MmusDv', # Mouse
                         'NCBITaxon:7955':'ZFS', # Zebrafish
                         'other':'HsapDv' # For all other organisms, use HsapDv
                        },
    'tissue':{'NCBITaxon:6239':'WBbt', # C. Elegans
              'NCBITaxon:7227':'FBbt', # Drosophila
              'NCBITaxon:7955':'ZFA', # Zebrafish
              'other':'UBERON' # For all other organisms, use UBERON
             }
}


for col in col_ont_map:
    map_dict = {}
    for label in sample_df[col].unique():
        if col == 'disease' and label == 'normal': # normal is not in MONDO ontology
            map_dict[label] = 'PATO:0000461'
        elif label in ['unknown','n/a']: # unknown and n/a won't be in ontologies, pass along
            map_dict[label] = label
        elif col in ['tissue','development_stage']:
            if col == 'tissue':
                # Find what tissue type is at label row
                if sample_df.loc[sample_df[col] == label, 'tissue_type'].tolist()[0] != 'tissue':
                    map_dict[label] = label # Don't map cell type in tissue
                    continue
            # Find what organism term id is at label row
            org_term_id = sample_df.loc[sample_df[col] == label, 'organism_ontology_term_id'].tolist()[0]
            if org_term_id in col_ont_map[col]:
                # Get ontology of specific organism and map label
                species_ont = col_ont_map[col][sample_df.loc[org_term_id]]
                term_id = ontology_parser.get_term_id_by_label(label, species_ont)
            else:
                term_id = ontology_parser.get_term_id_by_label(label, col_ont_map[col]['other'])
        else:
            term_id = ontology_parser.get_term_id_by_label(label, col_ont_map[col])
        if term_id == None:
            print(f"Matching '{col_ont_map[col]}' term id not found for label '{label}' in column '{col}'")
        map_dict[label] = term_id
    sample_df[col + '_ontology_term_id'] = sample_df[col].map(map_dict)
    del sample_df[col]
    
# Blank fields in worksheet result in NaN values in dataframe, replacing these with n/a ? 
# Could also replace with unknown for certain columns using fillna options?
sample_df.fillna('n/a', inplace=True)
sample_df

In [ ]:
bucket_name = 'czi-psomagen'
lab = 'marson'
proj = 'mapping-grns-perturb-seq'

In [ ]:
my_dir = f'{lab}-{proj}/'
r = s3client.list_objects(Bucket=bucket_name, Prefix=my_dir, Delimiter='/')
orders = [o['Prefix'].replace(my_dir,'') for o in r['CommonPrefixes']]

samples = []
for o in orders:
    r = s3client.list_objects(Bucket=bucket_name, Prefix=f'{my_dir}{o}', Delimiter='/')
    libs = [l['Prefix'].replace(f'{my_dir}{o}','') for l in r['CommonPrefixes']]
    for l in libs:
        r = s3client.list_objects(Bucket=bucket_name, Prefix=f'{my_dir}{o}{l}processed/cellranger/', Delimiter='/')
        dates = [d['Prefix'].replace(f'{my_dir}{o}{l}processed/cellranger/','') for d in r['CommonPrefixes']]
        for d in dates:
            r = s3client.list_objects(Bucket=bucket_name, Prefix=f'{my_dir}{o}{l}processed/cellranger/{d}outs/per_sample_outs/', Delimiter='/')
            subs = [s['Prefix'].replace(f'{my_dir}{o}{l}processed/cellranger/{d}outs/per_sample_outs/','') for s in r['CommonPrefixes']]
            for s in subs:
                samples.append({
                    'order': o,
                    'library': l.rstrip('/'),
                    'date': d,
                    'sample': s.rstrip('/')
                })

In [ ]:
#identify which sample to curate
n = 0
s = samples[n]
s

In [ ]:
order = s['order']
lib = s['library']
run_date = s['date']
samp = s['sample']

### Download and read in h5 file
s_dir = f'{lab}-{proj}/{order}{lib}/processed/cellranger/{run_date}outs/per_sample_outs/{samp}'
download_files(s_dir)
adata = sc.read_10x_h5('sample_filtered_feature_bc_matrix.h5', gex_only=True)

### Track additional tab gids from Lattice wrangling sheet
guide_gid = "1589848458"
pooled_gid = "2009719915"

adata = custom_var_to_obs(adata)
adata = add_library_id(adata, sheet, pooled_gid, samp)

adata.obs['lab'] = lab
adata.obs['project'] = proj
adata.obs['sample_name'] = samp

adata.obs = adata.obs.merge(sample_df, on='sample_name', how='left').set_index(adata.obs.index)
adata.uns['organism_ontology_term_id'] = adata.obs['organism_ontology_term_id'].unique()[0]
adata.obs.drop(columns=['organism_ontology_term_id'],inplace=True)

crispr_df = gather_crispr()
adata.obs = adata.obs.merge(
    crispr_df, left_index=True, right_index=True, how='left'
).set_index(adata.obs.index)

metrics_df = gather_metrics(samp)
for c in metrics_df.columns:
    adata.obs[c] = metrics_df[c].values[0]

cxg_add_labels(adata)

### Startswith('mt-') works for human & mouse, more attn needed for other organisms
adata.var['mt'] = adata.var['feature_name'].str.lower().str.startswith('mt-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], inplace=True)

### Add guide schema metadata to adata.uns and adata.obs
adata = add_guide_metadata(adata, sheet, guide_gid)
adata = determine_perturbation_strategy(adata)

adata.write(filename=f'{samp}_curated.h5ad', compression='gzip')

for f in ['sample_filtered_feature_bc_matrix.h5','crispr_analysis.tar.gz','metrics_summary.csv']:
    os.remove(f)